In [ ]:
import xarray as xr
import xgcm
from matplotlib import pyplot as plt
import nc_time_axis
from pathlib import Path
import xnemogcm as xn
import numpy as np


In [ ]:
path   = "/data/dkamm/nemo_output/DINO/Stratification"

In [ ]:
datadir = Path(path)

In [ ]:
domcfg   = xn.open_domain_cfg(datadir=datadir)

# Reproducing the bathymetry of DINO in python

In [ ]:
def cosh_bathy(pdistLam: float, pminlam:float, pmaxlam: float, plam: float) -> float:
    """        
    pdistLam    :  Length scale of the slope   [degrees]
    pmaxLam     :  Maximum longitude/latitude  [degrees]
    pminLam     :  Minimum longitude/latitude  [degrees]
    pLam        :  Value of longitude/latitude [degrees]
    pcosh       :  Resulting tapering factor
    """
    znorm = 1 + np.exp((pminlam - pmaxlam) / pdistLam)
    pcosh = 1 - (
        np.exp((plam - pmaxlam) / pdistLam)
        + np.exp(-(plam - pminlam) / pdistLam)
    ) / znorm
    return(pcosh)

In [ ]:
def gauss_ring(
        pdistLam:   float,
        plam0:      float,
        pphi0:      float,
        prad:       float,
        plam:       xr.DataArray,
        pphi:       xr.DataArray,
        pdep_top:   float,
        pdep_bot:   xr.DataArray
    ) -> float:
    """        
    pdistLam    : Length scale of the slope               [degrees]
    plam0       : Longitude of the center                 [degrees]
    pphi0       : Latitude of the center                  [degrees]
    prad        : radius of the circle                    [degrees]
    pLam        : Longitude                               [degrees]
    pPhi        : Latitude                                [degrees]
    pdep_top    : depth of the gaussian bump              [meters]
    pdep_bot    : depth of the basin                      [meters]
    """
    zx = plam - plam0
    zy = (pphi - pphi0)
    zexp = (- zx**2 - zy**2 + 2 * prad * np.sqrt(zx**2 + zy**2) - prad**2) / pdistLam**2
    mask = (pdep_bot >= pdep_top) * 1
    pring   = mask * ((pdep_top - pdep_bot) * np.exp( zexp ) + pdep_bot) + (1. - mask) * pdep_bot
    return(pring)

In [ ]:
def xgauss_ring(
        pdistLam:   float,
        plam0:      float,
        pphi0:      float,
        prad:       float,
        pdep_bot:   float,
        pdep_top:   float,
        plam:       float,
        pphi:       float
    ) -> float:
    return(xr.apply_ufunc(gauss_ring, pdistLam, plam0, pphi0, prad, pdep_bot, pdep_top, plam, pphi))

## Cosh-shaped boundary

In [ ]:
zminlam =   domcfg.glamt.min()
zmaxlam =   domcfg.glamt.max()
zminphi =   domcfg.gphit.min()
zmaxphi =   domcfg.gphit.max()

In [ ]:
zdistlam = 3.0
zdistphi = np.cos(2 * np.pi * zmaxphi / 360.) * zdistlam

In [ ]:
zx = cosh_bathy(zdistlam, zminlam, zmaxlam, domcfg.glamt)
zy = cosh_bathy(zdistphi, zminphi, zmaxphi, domcfg.gphit)
pbathy = zx * zy * ( 4000. - 2000. ) + 2000

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,6))
zy.plot(ax=axs[0])
(1 - zx).plot(ax=axs[1])
(zx * zy).plot(ax=axs[2])

In [ ]:
pbathy.plot(x='glamt', y='gphit', figsize=(6,8))

## Flattening of the bathymetry in the channel with cosh

In [ ]:
zslp_cha = 1.5
zcha_min = -70.
zcha_max = -50

In [ ]:
ztaper = cosh_bathy(zslp_cha, zcha_min, zcha_max, domcfg.gphit.where(domcfg.gphit < zcha_max)).fillna(0.0)

In [ ]:
pbathy_flat1 = ztaper * pbathy.max() + pbathy * (1. - ztaper)

In [ ]:
pbathy_flat1.plot(x='glamt', y='gphit', figsize=(6,8))

## Flattening of the bathymetry in the channel with gaussian ridge

In [ ]:
zds_width   = 4.
zlam0       = (zcha_max + zcha_min) / 2
zrad        = (zcha_max - zcha_min) / 2
zdep_top    = 3000

In [ ]:
pbathy_n = gauss_ring(zds_width, zminlam, zlam0, zrad, domcfg.glamt, domcfg.gphit, zdep_top, pbathy_flat1)

In [ ]:
fig, axs = plt.subplots(1,3, sharey=True, figsize=(20,10))
pbathy_n.isel(x_c=0,y_c=slice(0,70)).plot(ax=axs[0], label='0')
pbathy_n.isel(x_c=1,y_c=slice(0,70)).plot(ax=axs[0], label='1')
pbathy_n.isel(x_c=2,y_c=slice(0,70)).plot(ax=axs[0], label='2')
pbathy_n.isel(x_c=3,y_c=slice(0,70)).plot(ax=axs[0], label='3')
pbathy_n.isel(x_c=4,y_c=slice(0,70)).plot(ax=axs[0], label='4')
axs[0].legend()
pbathy_flat1.isel(x_c=0,y_c=slice(0,70)).plot(ax=axs[1], label='0')
pbathy_flat1.isel(x_c=1,y_c=slice(0,70)).plot(ax=axs[1], label='1')
pbathy_flat1.isel(x_c=2,y_c=slice(0,70)).plot(ax=axs[1], label='2')
pbathy_flat1.isel(x_c=3,y_c=slice(0,70)).plot(ax=axs[1], label='3')
pbathy_flat1.isel(x_c=4,y_c=slice(0,70)).plot(ax=axs[1], label='4')
axs[1].legend()

pbathy_flat1.isel(x_c=0,y_c=slice(0,70)).plot(ax=axs[2], label='0')
pbathy_n.isel(x_c=0,y_c=slice(0,70)).plot(ax=axs[2], label='0')
axs[2].grid('fine')

## Linear transition from Scottia arc to western boundary (Periodic boundary conditions) 

In [ ]:
taper = (pbathy_n.x_c > int(zdistlam))*1.0 + (pbathy_n.x_c <= int(zdistlam)) * pbathy_n.x_c / zdistlam

In [ ]:
bathy = taper * pbathy_n + (1 - taper) * pbathy_flat1

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,8))
bathy.plot(x='glamt', y='gphit', ax=ax)
ax.set_aspect('equal', adjustable='box')

## Tampering the bathymetry to reach the bottom in the channel

In [ ]:
def tampered_exponential(x, wall_x, wall_left, width, zdistlam, taper_dist, taper_ratio): 
    if wall_left:
        ones    = (x >= wall_x + taper_dist) * 1.0 + (x < wall_x) * 1.0
        sigmoid = ((x <  wall_x + taper_dist) * (x > wall_x)) * 1.0
        sign    = 1.0
        mask    = (x < wall_x) * 1.0
    else:
        ones    = (x <= wall_x - taper_dist) * 1.0 + (x > wall_x) * 1.0
        sigmoid = ((x >  wall_x - taper_dist) * (x < wall_x)) * 1.0
        sign    = - 1.0
        mask    = (x > wall_x) * 1.0
        
    inflec  = wall_x + sign * taper_ratio * taper_dist
    step = xr.zeros_like(x) + ones + sigmoid * 1 / (1 + np.exp( - sign * ( x - inflec) / 0.5))

    znorm   = 1 + np.exp(-( width) / zdistlam)
    exp     = 1 - (np.exp(- sign * (x - wall_x) / zdistlam)) / znorm

    return (exp * (1 - step) + step) - mask
    #return step, exp

In [ ]:
channel1 = tampered_exponential(domcfg.gphit, zcha_min, True, 140., zdistlam, 9., 0.8)
channel2 = tampered_exponential(domcfg.gphit, zcha_max, False, 140., zdistlam, 9., 0.8)

In [ ]:
wall1 = tampered_exponential(domcfg.gphit, zminphi, True, 140., zdistlam, 9., 0.8)
wall2 = tampered_exponential(domcfg.gphit, zmaxphi, False, 140., zdistlam, 9., 0.8)
wall3 = tampered_exponential(domcfg.glamt, zminlam, True, 140., zdistlam, 9., 0.8)
wall4 = tampered_exponential(domcfg.glamt, zmaxlam, False, 140., zdistlam, 9., 0.8)

channel1 = tampered_exponential(domcfg.gphit, zcha_min, True, 140., zdistlam, 9., 0.8)
channel2 = tampered_exponential(domcfg.gphit, zcha_max, False, 140., zdistlam, 9., 0.8)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(15, 6))
(wall1 * wall2).isel(x_c=30).plot(ax=axs[0])
(wall3 * wall4).isel(y_c=100).plot(ax=axs[1])
(channel2).isel(x_c=30).plot(ax=axs[2])

In [ ]:
wall3_new = channel2 + wall3 * (1. - channel2)
wall4_new = channel2 + wall4 * (1. - channel2)

In [ ]:
taper = wall1 * wall2 * wall3_new * wall4_new

In [ ]:
bathy = taper * ( 4000. - 2000. ) + 2000

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,8))
bathy.plot(x='glamt', y='gphit', ax=ax)
ax.set_aspect('equal', adjustable='box')

In [ ]:
bathy_ring = gauss_ring(zds_width, zminlam, zlam0, zrad, domcfg.glamt, domcfg.gphit, zdep_top, bathy)

In [ ]:
taper = (bathy_ring.x_c > int(zdistlam))*1.0 + (bathy_ring.x_c <= int(zdistlam)) * bathy_ring.x_c / zdistlam

In [ ]:
dino = taper * bathy_ring + (1 - taper) * bathy

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,8))
dino.plot(x='glamt', y='gphit', ax=ax)
ax.set_aspect('equal', adjustable='box')